In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,udf
from pyspark.sql.types import *

In [2]:
spark = SparkSession.builder.master("local[1]").appName("SparkByExamples.com").getOrCreate()  
spark.conf.set("spark.sql.execution.arrow.enabled", "true")
spark.conf.set("spark.sql.crossJoin.enabled", "true")

In [3]:
df=spark.read.parquet('DFText_total_entities_total.parquet')

In [4]:
df.select('UID').toPandas()

,UID
0,15-15-44943844163948
1,20-15-104697399642118
2,14-15-31075943189328
3,21-15-1205849911243790
4,07-2-6138959104
...,...
10363,16-4-59567975606808
10364,16-4-54515654971709
10365,20-4-1105178210203045
10366,16-4-52713884809039


In [5]:
#df.select('Entidades').toPandas()

In [6]:
df_1=spark.read.parquet('DFText_total_entities1.parquet')
df_2=spark.read.parquet('DFText_total_entities2.parquet')
df_3=spark.read.parquet('DFText_total_entities3.parquet')

In [7]:
l1=list([list(i)[0] for i in df_1.select('UID').toPandas().values])
l2=list([list(i)[0] for i in df_2.select('UID').toPandas().values])
l3=list([list(i)[0] for i in df_3.select('UID').toPandas().values])
ltotal=l1+l2+l3
len(ltotal)

7368

In [8]:
def select_new(uid):
    if uid in ltotal:
        res='s'
    else:
        res='n'
    return res
    

In [9]:
select_new_udf = udf(lambda z: select_new(z),StringType())

In [10]:
df=df.withColumn("Filter_1", select_new_udf(col("UID"))) 

In [11]:
df2=df.filter(df.Filter_1=='n')

In [12]:
df2.select('Entidades').count()

3000

In [14]:
DFparquet=pd.read_parquet('DFText_total_entities_total.parquet')

In [30]:
pd.DataFrame(DFparquet['Entidades'][0][1]).transpose()

,0,1
0,Confianza,"[0.8, 0.8, 0.8, 0.8, 0.8, 0.67, 0.88, 0.55, 0...."


In [40]:
DFparquet.dtypes

UID           object
Text          object
clean_text    object
Entidades     object
dtype: object

In [76]:
def func_2(entidades):
    dic_0=dict()
    for i in entidades:
        dic_0[i[0]]=i[1]
    return dic_0

In [79]:
DFparquet['Entidades']=DFparquet['Entidades'].apply(func_2)

In [46]:
DFparquet['Entidades'].apply(func_2)[0]['Subcategoría'].unique()

array(['Number', None, 'DateRange', 'Currency', 'Dimension', 'GPE',
       'Percentage'], dtype=object)

In [80]:
DFparquet.to_json('Total_data.json')

In [81]:
pd.read_json('Total_data.json')

,UID,Text,clean_text,Entidades
0,15-1-1484634171659,DO DE PERTENENCIA\n196\nENTINO\nMEJOR QUE PROM...,DO DE PERTENENCIA 196 ENTINO MEJOR QUE PROMESA...,"{'Compensación': ['18', '75', '79', '83', '88'..."
1,13-1-874921496833,ALCALDÍA DE\ntagůì\nEn Itagui se vive mejor.\n...,ALCALDÍA DE TAGŮÌ EN ITAGUI SE VIVE MEJOR. CON...,"{'Compensación': ['0', '12', '21', '97', '110'..."
2,14-1-1314883325605,REPUBLICA DE COLOMBIA\n*\n★\nDEPARTAMENTO DEL ...,REPUBLICA DE COLOMBIA * ★ DEPARTAMENTO DEL MAG...,"{'Compensación': ['0', '43', '66', '88', '133'..."
3,08-1-39609264810,TIM\nPLANOPENISE INTEGRADO DE MANGEALES S.A.\n...,TIM PLANOPENISE INTEGRADO DE MANGEALES S.A. CO...,"{'Compensación': ['0', '57', '59', '62', '169'..."
4,18-1-1955887908639,CÓDIGO: A1F005\nDEPARTAMENTO DE ANTIOQUIA\nMUN...,CÓDIGO: A1F005 DEPARTAMENTO DE ANTIOQUIA MUNIC...,"{'Compensación': ['15', '41', '77', '102', '12..."
...,...,...,...,...
10363,16-4-59567975606808,REPUBLICA DE COLOMBIA\nDEPARTAMENTO DEL TOLIMA...,REPUBLICA DE COLOMBIA DEPARTAMENTO DEL TOLIMA ...,"{'Compensación': ['0', '22', '152', '218', '24..."
10364,16-4-54515654971709,Municipio de San Benito Abad\n&TY\nNIT 8922800...,MUNICIPIO DE SAN BENITO ABAD &TY NIT 892280054...,"{'Compensación': ['0', '17', '37', '48', '51',..."
10365,20-4-1105178210203045,HSL\nHOSPITAL SANTA LUCÍA E.S.E.OH\nRONCESVALL...,HSL HOSPITAL SANTA LUCÍA E.S.E.OH RONCESVALLES...,"{'Compensación': ['0', '34', '60', '73', '99',..."
10366,16-4-52713884809039,"CONTRATO Nº 10\nMANTENIMIENTO, ARREGLO DE CHAP...","CONTRATO Nº 10 MANTENIMIENTO, ARREGLO DE CHAPA...","{'Compensación': ['12', '15', '63', '89', '104..."
